In [2]:
import pandas as pd
import numpy as np


In [5]:
data = pd.read_csv('/home/code_wizard/coding_files/machine_learning_1/Assignment_dataset.csv')
data.columns

Index(['Categorical_1', 'Categorical_2', 'Feature_1', 'Feature_2', 'Feature_3',
       'Label'],
      dtype='object')

In [ ]:
for feature in data.columns:
    data[feature]= ()